In [14]:
import string

import sys

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split

from torch.utils.data import Dataset, DataLoader, random_split

import torch

import torch.nn as nn

from sklearn.metrics import accuracy_score


sys.path.append("/kaggle/input/charactertokenizer/transformers/default/1")

from core import CharacterTokenizer



chars = "АаБбВвГгДдЕеЁёЖжЗзИиЙйКкЛлМмНнОоПпРрСсТтУуФфХхЦцЧчШшЩщЪъЫыЬьЭэЮюЯя"

model_max_length = 64

tokenizer = CharacterTokenizer(chars, model_max_length)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Задание: обучите модель классификации букв для задачи расстановки ударения с помощью методов из библиотеки transformers. Датасет для обучения можно взять отсюда: https://github.com/Koziev/NLP_Datasets/blob/master/Stress/all_accents.zip



1. Напишите класс для Dataset/Dataloder и разбейте данные на случайные train / test сплиты в соотношении 50:50. (1 балл)

2. Попробуйте обучить одну или несколько из моделей: Bert, Albert, Deberta. Посчитайте метрику Accuracy на train и test. (1 балл). При преодолении порога в Accuracy на test 0.8: (+1 балл), 0.85: (+2 балла), 0.89: (+3 балла).

Пример конфигурации для deberta: https://huggingface.co/IlyaGusev/ru-word-stress-transformer/blob/main/config.json

In [15]:
df = pd.read_table('/kaggle/input/all-accents/all_accents.tsv', header=None, names = ['word', 'stressed_word'])

df['stress_idx'] = df['stressed_word'].str.find('^')



train_df, test_df = train_test_split(df, test_size=0.5)



In [3]:
train_df

,word,stressed_word,stress_idx
1094902,помышлявшее,помышл^явшее,6
726284,нанятое,н^анятое,1
1089385,получающий,получ^ающий,5
631909,лыжницами,л^ыжницами,1
1031070,погрузившими,погруз^ившими,6
...,...,...,...
653770,машинальному,машин^альному,5
1295809,рассовавшие,рассов^авшие,6
1182500,прифабриваться,приф^абриваться,4
795115,номинальнейшем,номин^альнейшем,5


In [17]:
class WordStressDataset(Dataset):

    def __init__(self, df, max_len):

        self.df = df

        self.max_len = max_len

        self.tokenizer = CharacterTokenizer(chars, model_max_length)



    def __len__(self):

        return len(self.df)



    def __getitem__(self, idx):

        word = self.df['word'].iloc[idx]

        stress_idx = self.df['stress_idx'].iloc[idx]



        tokens = self.tokenizer(

            word,

            max_length=self.max_len,

            padding='max_length',

            truncation=True,

            return_tensors='pt'

        )

        labels = torch.zeros((self.max_len), dtype=torch.long)

        if stress_idx > 0:

            labels[stress_idx] = 1

        

        return tokens['input_ids'].flatten(), tokens['attention_mask'].flatten(), labels





train_dataset = WordStressDataset(train_df, model_max_length)

test_dataset = WordStressDataset(test_df, model_max_length)



train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataloader = DataLoader(test_dataset, batch_size=64)

In [18]:
from transformers import DebertaV2ForTokenClassification, DebertaV2Config,  AdamW



config = DebertaV2Config(

    architectures="DebertaV2ForTokenClassification",

    model_type="deberta-v2",

    vocab_size=len(tokenizer.get_vocab()),  # Размер словаря (включая специальные токены)

    torch_dtype="float32",


    hidden_size=768,  # Размер скрытого слоя

    num_hidden_layers=5,  # Количество скрытых слоёв

    num_attention_heads=12,  # Количество голов внимания


    intermediate_size=1024,  # Размер промежуточного слоя

    hidden_act="gelu",  # Функция активации для скрытых слоёв

    hidden_dropout_prob=0.15,  # Вероятность dropout для скрытых слоёв

    attention_probs_dropout_prob=0.15,  # Вероятность dropout для вероятностей внимания

    max_position_embeddings=model_max_length,  # Максимальная длина последовательности

    #type_vocab_size=1,  # Количество типов токенов (для сегментации)

    #initializer_range=0.02,  # Диапазон инициализации весов

    #layer_norm_eps=1e-7,  # Точность нормализации слоёв

    #pad_token_id=0,  # ID токена для заполнения

    #position_embedding_type="absolute",  # Тип позиционного вложения

    #use_cache=True,  # Использовать кеш для ускорения вычислений


    num_labels=2,  # Количество классов для задачи классификации (0 - без ударения, 1 - с ударением)

)



model = DebertaV2ForTokenClassification(config)

optimizer = torch.optim.AdamW(model.parameters(),lr = 1e-5,eps = 1e-8)

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
from tqdm import tqdm



def accuracy(preds, labels):    

    return np.all(preds == labels, axis=1).sum() / len(labels)



def train(train_dataloader,epoch):

    total_train_loss = 0

    total_train_acc=0

    pbar = tqdm(train_dataloader, desc=f"Epoch {epoch}")

    for step, batch in enumerate(pbar):

        b_input_ids = batch[0].to(device)

        b_input_mask = batch[1].to(device)

        b_labels = batch[2].to(device)
        
        model.to(device)

        model.zero_grad()

        outputs = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels)

        loss = outputs.loss

        total_train_loss += loss.item()

        logits = torch.argmax(outputs.logits.detach(), dim=2).cpu().numpy()

        label_ids = b_labels.cpu().numpy()

        total_train_acc += accuracy_score(logits, label_ids)

        loss.backward()

        optimizer.step()


    avg_train_loss = total_train_loss / len(train_dataloader)

    train_acc = total_train_acc/len(train_dataloader)

    print("epoch:", epoch)

    print("Average training loss: ",avg_train_loss)

    print("Train Accuracy: ", train_acc)

    return avg_train_loss, train_acc





def validate(val_dataloader, epoch):

    total_eval_accuracy = 0

    total_eval_loss = 0

    pbar = tqdm(val_dataloader, desc=f"Epoch {epoch}")

    for batch in pbar:

        b_input_ids = batch[0].to(device)

        b_input_mask = batch[1].to(device)

        b_labels = batch[2].to(device)

        with torch.no_grad():

            outputs = model(b_input_ids,attention_mask=b_input_mask,labels=b_labels)

        loss = outputs.loss

        total_eval_loss += loss.item()

        logits = torch.argmax(outputs.logits.detach(), dim=2).cpu().numpy()

        label_ids = b_labels.cpu().numpy()

        total_eval_accuracy += accuracy_score(logits, label_ids)



    avg_val_accuracy = total_eval_accuracy / len(val_dataloader)

    print("Validation Accuracy: ",avg_val_accuracy)

    avg_val_loss = total_eval_loss / len(val_dataloader)

    print("Validation Loss: ",avg_val_loss)

    

    return avg_val_loss, avg_val_accuracy

In [20]:
for i in range(0, 6):

    model.train()

    avg_train_loss = train(train_dataloader,i)

    model.eval()

    avg_test_loss, avg_test_accuracy = validate(test_dataloader, i)


Epoch 0: 100%|██████████| 13130/13130 [21:12<00:00, 10.31it/s]


epoch: 0
Average training loss:  0.020083255425410493
Train Accuracy:  0.6137035458353528


Epoch 0: 100%|██████████| 13130/13130 [10:28<00:00, 20.88it/s]


Validation Accuracy:  0.7549108276212236
Validation Loss:  0.014407553418375614


Epoch 1: 100%|██████████| 13130/13130 [21:13<00:00, 10.31it/s]


...

Validation Accuracy:  0.8846018976897689
Validation Loss:  0.007322115608377869


Epoch 5: 100%|██████████| 13130/13130 [21:10<00:00, 10.34it/s]


epoch: 5
Average training loss:  0.008070759062395664
Train Accuracy:  0.8665698781416603


Epoch 5: 100%|██████████| 13130/13130 [10:28<00:00, 20.90it/s]

Validation Accuracy:  0.8957904925107895
Validation Loss:  0.006660471626396752


In [21]:
for i in range(6, 8):

    model.train()

    avg_train_loss = train(train_dataloader,i)

    model.eval()

    avg_test_loss, avg_test_accuracy = validate(test_dataloader, i)


Epoch 6: 100%|██████████| 13130/13130 [21:18<00:00, 10.27it/s]


epoch: 6
Average training loss:  0.0073848336300832524
Train Accuracy:  0.8781061760022155


Epoch 6: 100%|██████████| 13130/13130 [10:28<00:00, 20.89it/s]


Validation Accuracy:  0.9054927487941101
Validation Loss:  0.005985709611816411


Epoch 7: 100%|██████████| 13130/13130 [21:12<00:00, 10.32it/s]


epoch: 7
Average training loss:  0.006806897448098481
Train Accuracy:  0.8878072422626877


Epoch 7: 100%|██████████| 13130/13130 [10:31<00:00, 20.79it/s]

Validation Accuracy:  0.9120438245747651
Validation Loss:  0.005601048181621679


In [22]:
print("Accuracy on test", avg_test_accuracy)

Accuracy on test 0.9120438245747651
